In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/oxford-nasa-rfud/RFUD_RW1_2_7_8.parquet
/kaggle/input/oxford-nasa-rfud/00006.csv
/kaggle/input/oxford-nasa-rfud/1.RFUD_RW9_10_11_12.parquet
/kaggle/input/oxford-nasa-rfud/RFUD_RW1_12_ALL_FINAL.parquet
/kaggle/input/oxford-nasa-rfud/B0018_numeric_interpolated.csv
/kaggle/input/oxford-nasa-rfud/B0018_numeric_raw.csv
/kaggle/input/oxford-nasa-rfud/OXFORD_checkpoint_features.parquet
/kaggle/input/oxford-nasa-rfud/00007.csv
/kaggle/input/oxford-nasa-rfud/RFUD_RW3_4_5_6.parquet
/kaggle/input/oxford-nasa-rfud/00005.csv


In [2]:
from pathlib import Path
import os

BASE_DIR = Path("/kaggle/input")

print("📂 Available datasets:")
for d in BASE_DIR.iterdir():
    print(" -", d.name)

print("\n📄 Listing files:")
for root, dirs, files in os.walk(BASE_DIR):
    for f in files:
        if f.endswith((".csv", ".parquet")):
            print(os.path.join(root, f))


📂 Available datasets:
 - oxford-nasa-rfud

📄 Listing files:
/kaggle/input/oxford-nasa-rfud/RFUD_RW1_2_7_8.parquet
/kaggle/input/oxford-nasa-rfud/00006.csv
/kaggle/input/oxford-nasa-rfud/1.RFUD_RW9_10_11_12.parquet
/kaggle/input/oxford-nasa-rfud/RFUD_RW1_12_ALL_FINAL.parquet
/kaggle/input/oxford-nasa-rfud/B0018_numeric_interpolated.csv
/kaggle/input/oxford-nasa-rfud/B0018_numeric_raw.csv
/kaggle/input/oxford-nasa-rfud/OXFORD_checkpoint_features.parquet
/kaggle/input/oxford-nasa-rfud/00007.csv
/kaggle/input/oxford-nasa-rfud/RFUD_RW3_4_5_6.parquet
/kaggle/input/oxford-nasa-rfud/00005.csv


In [3]:
CANONICAL_COLUMNS = [
    "asset_id",
    "cycle_id",
    "V_mean",
    "V_std",
    "V_min",
    "V_max",
    "dV_dt_mean",
    "dV_dt_max",
    "V_range",
    "T_mean",
    "T_max",
    "T_delta",
    "duration_s",
    "source"
]


In [4]:
import pandas as pd

oxford = pd.read_parquet(
    "/kaggle/input/oxford-nasa-rfud/OXFORD_checkpoint_features.parquet"
)

oxford = oxford.rename(columns={"cell_id": "asset_id"})
oxford = oxford[CANONICAL_COLUMNS]

print("✅ Oxford loaded:", oxford.shape)
oxford.head()


✅ Oxford loaded: (519, 14)


,asset_id,cycle_id,V_mean,V_std,V_min,V_max,dV_dt_mean,dV_dt_max,V_range,T_mean,T_max,T_delta,duration_s,source
0,0,0,3.735076,0.245422,2.699819,4.191235,-35.948592,38.111247,1.491416,40.419340,41.174809,1.273163,0.041740,oxford
1,0,1,3.735681,0.244972,2.699859,4.192679,-36.342517,34.652440,1.492819,40.327115,41.124866,1.323108,0.041245,oxford
2,0,2,3.736424,0.244468,2.700020,4.192959,-36.676399,34.643800,1.492939,40.327298,41.112400,1.298134,0.040997,oxford
3,0,3,3.736474,0.243927,2.699964,4.192502,-36.930236,45.049657,1.492538,40.322045,41.124866,1.335575,0.040834,oxford
4,0,4,3.737696,0.242782,2.700164,4.192782,-36.822717,34.646588,1.492618,40.314726,41.124866,1.335575,0.040577,oxford


In [5]:
nasa = pd.read_csv(
    "/kaggle/input/oxford-nasa-rfud/B0018_numeric_interpolated.csv"
)

nasa_features = (
    nasa.groupby("cycle")
    .agg(
        V_mean=("voltage", "mean"),
        V_std=("voltage", "std"),
        V_min=("voltage", "min"),
        V_max=("voltage", "max"),
        dV_dt_mean=("voltage", lambda x: x.diff().mean()),
        dV_dt_max=("voltage", lambda x: x.diff().max()),
        T_mean=("temperature", "mean"),
        T_max=("temperature", "max"),
        T_delta=("temperature", lambda x: x.max() - x.min()),
        duration_s=("time", lambda x: x.iloc[-1] - x.iloc[0]),
    )
    .reset_index()
    .rename(columns={"cycle": "cycle_id"})
)

nasa_features["asset_id"] = "nasa_B0018"
nasa_features["V_range"] = nasa_features["V_max"] - nasa_features["V_min"]
nasa_features["source"] = "nasa"

nasa_features = nasa_features[CANONICAL_COLUMNS]

print("✅ NASA loaded:", nasa_features.shape)
nasa_features.head()


KeyError: 'cycle'

In [ ]:
print(nasa.columns.tolist())


In [ ]:
import numpy as np
import pandas as pd

imp = nasa.copy()

imp_features = pd.DataFrame({
    "Z_real_mean": imp["Battery_impedance_real"].mean(),
    "Z_real_std": imp["Battery_impedance_real"].std(),
    "Z_imag_mean": imp["Battery_impedance_imag"].mean(),
    "Z_imag_std": imp["Battery_impedance_imag"].std(),

    "Z_rect_mean": imp["Rectified_Impedance_real"].mean(),
    "Z_rect_std": imp["Rectified_Impedance_real"].std(),

    "phase_proxy_mean": np.arctan2(
        imp["Battery_impedance_imag"],
        imp["Battery_impedance_real"]
    ).mean(),

    "magnitude_proxy_mean": np.sqrt(
        imp["Battery_impedance_real"]**2 +
        imp["Battery_impedance_imag"]**2
    ).mean()
}, index=[0])

imp_features["asset_id"] = "nasa_B0018_EIS"
imp_features["source"] = "nasa_eis"

imp_features


In [ ]:
imp_features.to_parquet(
    "/kaggle/working/NASA_EIS_features.parquet",
    index=False
)


In [ ]:
CANONICAL_COLUMNS = [
    "asset_id",
    "cycle_id",
    "V_mean", "V_std", "V_min", "V_max",
    "dV_dt_mean", "dV_dt_max",
    "V_range", "V_mid", "V_low", "V_high",
    "T_mean", "T_max", "T_delta",
    "duration_s",
    "source"
]


In [ ]:
CANONICAL_COLUMNS = [
    "asset_id",
    "cycle_id",
    "V_mean", "V_std", "V_min", "V_max",
    "dV_dt_mean", "dV_dt_max",
    "V_range", "V_mid", "V_low", "V_high",
    "T_mean", "T_max", "T_delta",
    "duration_s",
    "source"
]


In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
from tqdm import tqdm


In [ ]:
def extract_checkpoint_features(df, asset_prefix, source="rfud"):
    # Ensure ordering for dV/dt
    df = df.sort_values(["cycle_id", "time_s"])

    g = df.groupby("cycle_id")

    out = pd.DataFrame({
        "cycle_id": g.size().index,
        "V_mean": g["voltage_V"].mean(),
        "V_std": g["voltage_V"].std(),
        "V_min": g["voltage_V"].min(),
        "V_max": g["voltage_V"].max(),
        "dV_dt_mean": g["voltage_V"].apply(lambda x: x.diff().mean()),
        "dV_dt_max": g["voltage_V"].apply(lambda x: x.diff().max()),
        "T_mean": g["temperature_C"].mean(),
        "T_max": g["temperature_C"].max(),
        "T_delta": g["temperature_C"].apply(lambda x: x.max() - x.min()),
        "duration_s": g["time_s"].apply(lambda x: x.iloc[-1] - x.iloc[0]),
    }).reset_index(drop=True)

    out["V_range"] = out["V_max"] - out["V_min"]
    out["V_mid"]   = (out["V_max"] + out["V_min"]) / 2
    out["V_low"]   = out["V_min"] + 0.25 * out["V_range"]
    out["V_high"]  = out["V_min"] + 0.75 * out["V_range"]

    out["asset_id"] = asset_prefix
    out["source"] = source

    return out[CANONICAL_COLUMNS]


In [ ]:
RFUD_FILES = [
    "/kaggle/input/oxford-nasa-rfud/RFUD_RW1_2_7_8.parquet",
    "/kaggle/input/oxford-nasa-rfud/RFUD_RW3_4_5_6.parquet",
    "/kaggle/input/oxford-nasa-rfud/1.RFUD_RW9_10_11_12.parquet",
]


In [ ]:
import os
import pyarrow.parquet as pq
import pandas as pd

OUTPUT_DIR = "/kaggle/working/RFUD_checkpoint_parts"
os.makedirs(OUTPUT_DIR, exist_ok=True)

all_chunks = []
part_id = 0


In [ ]:
# 🔐 Enforce canonical dtypes (CRITICAL)
feats["cycle_id"] = feats["cycle_id"].astype(str)
feats["asset_id"] = feats["asset_id"].astype(str)
feats["source"] = feats["source"].astype(str)


In [ ]:
assert feats["cycle_id"].dtype == object


In [ ]:
CANONICAL_DTYPES = {
    "cycle_id": "string",
    "asset_id": "string",
    "source": "string",
}


In [ ]:
import pyarrow.parquet as pq
import pandas as pd

features = []

pf = pq.ParquetFile(RFUD_PATH)

for i in range(pf.num_row_groups):
    print(f"Processing row group {i+1}/{pf.num_row_groups}")

    # 1️⃣ df is created HERE
    df = pf.read_row_group(i).to_pandas()

    # 2️⃣ 🔐 DTYPE FIX MUST BE HERE
    df["cycle_id"] = df["cycle_id"].astype("string")

    # 3️⃣ aggregation
    grp = df.groupby("cycle_id").agg(
        V_mean=("voltage_V", "mean"),
        V_std=("voltage_V", "std"),
        V_min=("voltage_V", "min"),
        V_max=("voltage_V", "max"),
        dV_dt_mean=("voltage_V", lambda x: x.diff().mean()),
        dV_dt_max=("voltage_V", lambda x: x.diff().max()),
        T_mean=("temperature_C", "mean"),
        T_max=("temperature_C", "max"),
        T_delta=("temperature_C", lambda x: x.max() - x.min()),
        duration_s=("time_s", lambda x: x.iloc[-1] - x.iloc[0]),
    ).reset_index()

    grp["asset_id"] = ASSET_NAME
    grp["source"] = "rfud"

    features.append(grp)

    del df, grp


In [ ]:
import pyarrow.parquet as pq
import pandas as pd

RFUD_PATH = "/kaggle/input/oxford-nasa-rfud/RFUD_RW1_2_7_8.parquet"
ASSET_NAME = "RFUD_RW1_2_7_8"

features = []

pf = pq.ParquetFile(RFUD_PATH)


In [ ]:
RFUD_FILES = [
    "/kaggle/input/oxford-nasa-rfud/RFUD_RW1_2_7_8.parquet",
    "/kaggle/input/oxford-nasa-rfud/RFUD_RW3_4_5_6.parquet",
    "/kaggle/input/oxford-nasa-rfud/1.RFUD_RW9_10_11_12.parquet",
]


In [ ]:
import pandas as pd
import pyarrow.parquet as pq
from tqdm import tqdm
import os


In [ ]:
RFUD_FILES = [
    "/kaggle/input/oxford-nasa-rfud/RFUD_RW1_2_7_8.parquet",
    "/kaggle/input/oxford-nasa-rfud/RFUD_RW3_4_5_6.parquet",
    "/kaggle/input/oxford-nasa-rfud/1.RFUD_RW9_10_11_12.parquet",
]

OUTPUT_DIR = "/kaggle/working/rfud_checkpoints"
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [ ]:
part_id = 0

for RFUD_PATH in RFUD_FILES:
    ASSET_NAME = RFUD_PATH.split("/")[-1].replace(".parquet", "")
    print(f"\n🔄 Processing {ASSET_NAME}")

    pf = pq.ParquetFile(RFUD_PATH)
    features_buffer = []

    for i in tqdm(range(pf.num_row_groups)):
        df = pf.read_row_group(i).to_pandas()

        # 🔐 CRITICAL dtype fix (prevents Kaggle crash)
        df["cycle_id"] = df["cycle_id"].astype("string")

        grp = (
            df.groupby("cycle_id")
            .agg(
                V_mean=("voltage_V", "mean"),
                V_std=("voltage_V", "std"),
                V_min=("voltage_V", "min"),
                V_max=("voltage_V", "max"),
                dV_dt_mean=("voltage_V", lambda x: x.diff().mean()),
                dV_dt_max=("voltage_V", lambda x: x.diff().max()),
                T_mean=("temperature_C", "mean"),
                T_max=("temperature_C", "max"),
                T_delta=("temperature_C", lambda x: x.max() - x.min()),
                duration_s=("time_s", lambda x: x.iloc[-1] - x.iloc[0]),
            )
            .reset_index()
        )

        grp["asset_id"] = ASSET_NAME
        grp["source"] = "rfud"
        grp["V_range"] = grp["V_max"] - grp["V_min"]

        features_buffer.append(grp)

        del df, grp

        # 🔄 Flush periodically to disk (RAM safe)
        if len(features_buffer) >= 8:
            out_path = f"{OUTPUT_DIR}/rfud_part_{part_id:04d}.parquet"
            pd.concat(features_buffer).to_parquet(out_path, index=False)
            features_buffer.clear()
            part_id += 1

    # final flush per file
    if features_buffer:
        out_path = f"{OUTPUT_DIR}/rfud_part_{part_id:04d}.parquet"
        pd.concat(features_buffer).to_parquet(out_path, index=False)
        features_buffer.clear()
        part_id += 1


In [ ]:
import glob
import pandas as pd

rfud_parts = sorted(glob.glob("/kaggle/working/rfud_checkpoints/*.parquet"))
print("RFUD parts:", len(rfud_parts))

rfud_df = pd.concat(
    [pd.read_parquet(p) for p in rfud_parts],
    ignore_index=True
)

print("RFUD shape:", rfud_df.shape)
rfud_df.head()


In [ ]:
oxford_df = pd.read_parquet(
    "/kaggle/input/oxford-nasa-rfud/OXFORD_checkpoint_features.parquet"
)

oxford_df = oxford_df.rename(columns={"cell_id": "asset_id"})
oxford_df["asset_id"] = "oxford_" + oxford_df["asset_id"].astype(str)

print("Oxford shape:", oxford_df.shape)


In [ ]:
import pandas as pd

nasa_raw = pd.read_csv(
    "/kaggle/input/oxford-nasa-rfud/B0018_numeric_interpolated.csv"
)

print(nasa_raw.columns)
nasa_raw.head()


In [ ]:
import pandas as pd
import numpy as np

eis = pd.read_csv(
    "/kaggle/input/oxford-nasa-rfud/B0018_numeric_interpolated.csv"
)

eis.head()


In [ ]:
nasa_eis_features = pd.DataFrame([{
    "Z_real_mean": eis["Battery_impedance_real"].mean(),
    "Z_real_std": eis["Battery_impedance_real"].std(),
    "Z_imag_mean": eis["Battery_impedance_imag"].mean(),
    "Z_imag_std": eis["Battery_impedance_imag"].std(),
    "Z_rect_mean": eis["Rectified_Impedance_real"].mean(),
    "Z_rect_std": eis["Rectified_Impedance_real"].std(),

    # phase & magnitude proxies
    "phase_proxy_mean": np.mean(
        np.arctan2(
            eis["Battery_impedance_imag"],
            eis["Battery_impedance_real"]
        )
    ),
    "magnitude_proxy_mean": np.mean(
        np.sqrt(
            eis["Battery_impedance_real"]**2 +
            eis["Battery_impedance_imag"]**2
        )
    ),

    "asset_id": "nasa_B0018_EIS",
    "source": "nasa_eis"
}])

nasa_eis_features


In [ ]:
import pandas as pd
import numpy as np

# Load NASA EIS data
eis_df = pd.read_csv(
    "/kaggle/input/oxford-nasa-rfud/B0018_numeric_interpolated.csv"
)

print("Shape:", eis_df.shape)
eis_df.head()


In [ ]:
print(eis_df.columns)


In [ ]:
# Extract real & imaginary impedance
Z_real = eis_df["Battery_impedance_real"]
Z_imag = eis_df["Battery_impedance_imag"]

# Magnitude & phase proxies
Z_mag = np.sqrt(Z_real**2 + Z_imag**2)
Z_phase = np.arctan2(Z_imag, Z_real)

eis_features = pd.DataFrame({
    "Z_real_mean": [Z_real.mean()],
    "Z_real_std":  [Z_real.std()],
    "Z_imag_mean": [Z_imag.mean()],
    "Z_imag_std":  [Z_imag.std()],
    "Z_rect_mean": [eis_df["Rectified_Impedance_real"].mean()],
    "Z_rect_std":  [eis_df["Rectified_Impedance_real"].std()],
    "phase_proxy_mean": [Z_phase.mean()],
    "magnitude_proxy_mean": [Z_mag.mean()],
})


In [ ]:
eis_features["asset_id"] = "nasa_B0018_EIS"
eis_features["source"] = "nasa_eis"

eis_features


In [ ]:
EIS_OUT_PATH = "/kaggle/working/NASA_EIS_features.parquet"

eis_features.to_parquet(EIS_OUT_PATH, index=False)

print("✅ EIS features saved:", EIS_OUT_PATH)


In [ ]:
CANONICAL_COLUMNS = [
    "asset_id",
    "cycle_id",

    # Voltage
    "V_mean",
    "V_std",
    "V_min",
    "V_max",
    "V_range",

    # Dynamics
    "dV_dt_mean",
    "dV_dt_max",

    # Thermal
    "T_mean",
    "T_max",
    "T_delta",

    # Capacity / duration
    "Q_total",
    "duration_s",

    # Metadata
    "source"
]


In [ ]:
import pandas as pd

oxford = pd.read_parquet(
    "/kaggle/input/oxford-nasa-rfud/OXFORD_checkpoint_features.parquet"
)

oxford = oxford.rename(columns={"cell_id": "asset_id"})
oxford["asset_id"] = oxford["asset_id"].astype(str)
oxford["cycle_id"] = oxford["cycle_id"].astype(str)

# Oxford has no Q_total
oxford["Q_total"] = pd.NA

oxford = oxford[CANONICAL_COLUMNS]

print("✅ Oxford:", oxford.shape)


In [ ]:
import pyarrow.parquet as pq
from tqdm import tqdm
import os

RFUD_FILES = [
    "/kaggle/input/oxford-nasa-rfud/RFUD_RW1_2_7_8.parquet",
    "/kaggle/input/oxford-nasa-rfud/RFUD_RW3_4_5_6.parquet",
    "/kaggle/input/oxford-nasa-rfud/1.RFUD_RW9_10_11_12.parquet",
]

rfud_features = []

for path in RFUD_FILES:
    asset_prefix = path.split("/")[-1].replace(".parquet", "")
    pf = pq.ParquetFile(path)

    print(f"\n🔄 Processing {asset_prefix}")

    for rg in tqdm(range(pf.num_row_groups)):
        df = pf.read_row_group(rg).to_pandas()

        # 🔐 dtype fix (THIS avoids Kaggle crash)
        df["cycle_id"] = df["cycle_id"].astype(str)

        feats = (
            df.groupby("cycle_id")
            .agg(
                V_mean=("voltage_V", "mean"),
                V_std=("voltage_V", "std"),
                V_min=("voltage_V", "min"),
                V_max=("voltage_V", "max"),
                dV_dt_mean=("voltage_V", lambda x: x.diff().mean()),
                dV_dt_max=("voltage_V", lambda x: x.diff().max()),
                T_mean=("temperature_C", "mean"),
                T_max=("temperature_C", "max"),
                T_delta=("temperature_C", lambda x: x.max() - x.min()),
                duration_s=("time_s", lambda x: x.iloc[-1] - x.iloc[0]),
            )
            .reset_index()
        )

        feats["V_range"] = feats["V_max"] - feats["V_min"]
        feats["Q_total"] = pd.NA
        feats["asset_id"] = asset_prefix
        feats["source"] = "rfud"

        rfud_features.append(feats[CANONICAL_COLUMNS])

        del df


In [ ]:
rfud = pd.concat(rfud_features, ignore_index=True)

print("✅ RFUD:", rfud.shape)


In [ ]:
canonical = pd.concat([oxford, rfud], ignore_index=True)

print("🧠 CANONICAL FEATURE STORE")
print(canonical.shape)
canonical.head()


In [ ]:
OUT_PATH = "/kaggle/working/canonical_checkpoint_features.parquet"

canonical.to_parquet(
    OUT_PATH,
    engine="pyarrow",
    compression="snappy",
    index=False
)

print("✅ Saved:", OUT_PATH)
print("Size (MB):", round(os.path.getsize(OUT_PATH) / 1e6, 2))


In [ ]:
canonical.isna().mean().sort_values(ascending=False)
